In [56]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, confusion_matrix

np.random.seed(42)  # You can use any integer value
random_state = 42

In [57]:
# import "C:/Projects/public/XAI_Master/datasets/red1green0.csv"

red1green0 = pd.read_csv("C:/Projects/public/XAI_Master/datasets/red1green0.csv",
    header=0,          # Use first row as headers
    index_col=None,    # Don't use any column as index
    float_precision='high'  # For precise float values
    )
red0green1 = pd.read_csv("C:/Projects/public/XAI_Master/datasets/red0green1.csv",
    header=0,
    index_col=None,
    float_precision='high'
    )

# Merge and shuffle the two datasets
df = pd.concat([red1green0, red0green1])
df = df.sample(frac=1, random_state=random_state).reset_index(drop=True)
"""
df = red0green1
"""
x = df.drop('target', axis=1)
#x = df.drop(['target',"agentX","agentY","redX","redY","greenX","greenY"], axis=1)
print(x.shape)
y = df['target']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=random_state)

(10000, 134)


In [58]:
# Train and make predictions
model = LinearRegression()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

# Print float results
print(f"Mean Squared Error: {mean_squared_error(y_test, y_pred):.4f}")
print(f"R2 Score: {r2_score(y_test, y_pred):.4f}")

# Round y_pred to int, max 1 and min -1
y_pred = np.clip(np.round(y_pred), -1, 1)

# Print int results
total_correct = (y_test == y_pred).sum()
total_samples = len(y_test)
print(f"Total correct predictions: {total_correct} out of {total_samples}")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.2%}")
print(f"\nConfusion matrix:\n{confusion_matrix(y_test, y_pred)}")


Mean Squared Error: 0.1268
R2 Score: 0.7989
Total correct predictions: 1627 out of 2000
Accuracy: 81.35%

Confusion matrix:
[[1008   30    0]
 [   5  455   86]
 [   0  252  164]]


In [59]:
# Train and make predictions
tree = DecisionTreeClassifier(random_state=42)
tree.fit(x_train, y_train)
y_pred = tree.predict(x_test)

# Print results
total_correct = (y_test == y_pred).sum()
total_samples = len(y_test)
print(f"Total correct predictions: {total_correct} out of {total_samples}")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.2%}")
print(f"\nConfusion matrix:\n{confusion_matrix(y_test, y_pred)}")

Total correct predictions: 1998 out of 2000
Accuracy: 99.90%

Confusion matrix:
[[1037    1    0]
 [   0  545    1]
 [   0    0  416]]


A baseline is established. Between one good red agent and one week green agent, the decision tree managed to predict 78.4% of the cases correctly based on only neuron activations from, fc1, the second last layer with 128 neurons. When also giving the tree the coordinates for the agent, red and greens chests, the tree performed much better, only failing at 2-6 of 2000 test cases. (Depending on random seed)

My thoughts now is that I wonder how much the decision tree learns to identify the neural net and therefore knows if it is a good or bad neural net. I'm also thinking that I need many more neural nets to balance out this, especially very diffferent ones.
But probably, the next step is to try with a XAI method instead of activa
tions, since it is this comparison which is the most important part of the thesis. Maybe gradcam or shap. SHAP is probably good since it yield a map of what the agent "focuses" on. It is also a great baseline for later. So, in short, do shap, and more models.